In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt

SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]


df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df_primary = pd.read_csv("../../db/data/primary/2022/closes.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df_primary.index = pd.to_datetime(df_primary.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 9, 7, 0, 0, 0, 0, UTC)]
df_primary =  df_primary.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 9, 7, 0, 0, 0, 0, UTC)]


In [6]:
def mean_volatility_prediction(logs, rate=120):
    data = pd.DataFrame([])
    for ccy in logs.columns:
        data[ccy] = logs[ccy].rolling(rate).mean() + logs[ccy].rolling(rate).std()*2
    data.index = logs.index
    data2 = data.iloc[-1:].copy()

    return data, data2
    
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    data = data.iloc[-1:]
    return data

tend_pred = random_prediction(df)
last_vol, pred_vol = mean_volatility_prediction(df)

def condition(x):
    if x>0:
        return 1
    elif x< 0.5:
        return -1
    else:
        return 0

for sym in tend_pred.columns:
    tend_pred[sym] = tend_pred[sym].map( lambda x: condition(x))






In [11]:
def correlation_pairs(df, rate=1):

    data = pd.DataFrame(index=df.index)
    for pair in df.columns:
        for second_pair in df.columns:
            data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
            
    data.index = df.index
    data = data.iloc[-1:].T
    data.columns = ["correlation"]
    data = data[round(data["correlation"],6) != 1.000000]
    data = data.T
    
    return data

def weighted_volatility(volatility):
    weight = pd.DataFrame([])
    volatility = volatility.copy()
    volatility_T = volatility.T
    weight = volatility_T/ volatility_T.mean()
    return  weight

def expected_volatility(pred_vol, correlations):
    
    vols = pred_vol.copy()
    pred_vol = abs(pred_vol)
    
    
    for sym in pred_vol.columns:
        
        real_vol = 0
        for sym2 in pred_vol.columns:
            
            if sym == sym2:
                real_vol += pred_vol[sym]
                
            else:
                real_vol += pred_vol[sym2] * correlations.filter(regex=sym+'_'+sym2).values[0][0]
                
        vols[sym] = abs(round(real_vol.mean(),6))
        
    total_vol = vols.mean(axis=1)[0]
    

    return total_vol, vols

def trade_signals(pred_vols,last_data, classification, exchange_rate, risk=1, balance=100000, leverage=1):
    cor = correlation_pairs(last_data)
        
    t,v = expected_volatility(pred_vols, cor)
    n = int((abs(classification.T)).sum())
    risked = risk*balance*leverage/(t*n)
 
    
    trade_orders = []
    trade_dataframe = {"datetime":[],"currency":[], "side":[], "size":[]}
    for ccy in classification.columns:
        d = {}
        if int(classification[ccy]) != 0:
            d["currency"] = ccy
            trade_dataframe["currency"].append(ccy)
            signal = "sell" if int(classification[ccy]) == -1 else "buy"
            trade_dataframe["side"].append(signal)
            
            if ccy.split("_")[0] == "USD":
                d["size"] = round(risked*int(classification[ccy]))
                trade_dataframe["size"].append(round(risked))
            else:
                d["size"] = round(risked*float(exchange_rate[ccy.split("_")[0]])*int(classification[ccy]))
                trade_dataframe["size"].append(round(risked*float(exchange_rate[ccy.split("_")[0]])))
            d["datetime"] = v.index.to_list()[0].strftime("%Y-%m-%dT%H:%M:%S.Z")
            trade_dataframe["datetime"].append(v.index.to_list()[0])
            
            trade_orders.append(d)
    
    trade_dataframe = pd.DataFrame.from_dict(trade_dataframe)
    trade_dataframe.set_index("datetime",inplace=True)
    trade_dataframe.index = pd.to_datetime(trade_dataframe.index)

    return trade_orders, trade_dataframe 

def read_exchange_rate(df):
    
    rate = df.iloc[-1:]
    
    data = pd.DataFrame(index=rate.index)
    
    for ccy in rate.columns:
        if ccy.split("_",2)[0] == "USD":
            data[(ccy.split("_")[1])] = rate[ccy]
        elif ccy.split("_",2)[1] == "USD":
            data[(ccy.split("_")[0])] = 1/rate[ccy]
        
    return data
            
    

In [12]:
exchange_rate = read_exchange_rate(df_primary)
trade_signal, trade_df = trade_signals(pred_vol,last_vol, tend_pred, exchange_rate, risk=1, balance=100000, leverage=1)      

/tmp/ipykernel_5329/2486507700.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_5329/2486507700.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_5329/2486507700.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [13]:
trade_df
trade_signal


,currency,side,size
datetime,,,
2022-09-07 00:00:00+00:00,AUD_CAD,sell,1290
2022-09-07 00:00:00+00:00,AUD_CHF,buy,1290
2022-09-07 00:00:00+00:00,AUD_JPY,buy,1290
2022-09-07 00:00:00+00:00,AUD_NZD,sell,1290
2022-09-07 00:00:00+00:00,AUD_USD,sell,1290
2022-09-07 00:00:00+00:00,CAD_CHF,sell,1142
2022-09-07 00:00:00+00:00,CAD_JPY,buy,1142
2022-09-07 00:00:00+00:00,CHF_JPY,sell,854
2022-09-07 00:00:00+00:00,EUR_AUD,sell,876


In [21]:
from upload_gbq import upload_trade_signals_dataframe

In [22]:
upload_trade_signals_dataframe(trade_df, 'trade_signals')

Dataset artful-talon-355716.rex_ai exists
No data found in artful-talon-355716.rex_ai.trade_signals
Uploading last 28 hours from trade_signals to artful-talon-355716.rex_ai
Uploaded successfully in 5.52s


In [10]:
def trade_signals(pred_vols,last_data, classification, exchange_rate, risk=0.01, balance=100000, leverage=1):
    cor = correlation_pairs(last_data)
    
    t,v = expected_volatility(pred_vols, cor)
    n = int((abs(classification.T)).sum())
    risked = risk*balance*leverage/(t*n)
 
    
    trade_signals = []
    trade_dataframe = {"index":[v.index.to_list()[0]]}
     
    for ccy in classification.columns:
        d = {}
                
        if int(classification[ccy]) != 0:
            d["currency"] = ccy
            signal = "sell" if int(classification[ccy]) == -1 else "buy"
            d["side"] = signal
            trade_dataframe[f"{ccy}_side"] = signal
            if ccy.split("_")[0] == "USD":
                d["size"] = round(risked)
                trade_dataframe[f"{ccy}_size"] = round(risked)
            else:
                d["size"] = round(risked*float(exchange_rate[ccy.split("_")[0]]))
                trade_dataframe[f"{ccy}_size"] = round(risked*float(exchange_rate[ccy.split("_")[0]]))
            d["datetime"] = v.index.to_list()[0].strftime("%Y-%m-%dT%H:%M:%S.Z")
            
            
            trade_signals.append(d)
    trade_dataframe = pd.DataFrame.from_dict(trade_dataframe)
    trade_dataframe.set_index("index", inplace=True)
    trade_dataframe.index = pd.to_datetime(trade_dataframe.index)

    return trade_signals, trade_dataframe 